In [ ]:
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, MaxPooling2D, MaxPool2D,\
    BatchNormalization,Activation,Dropout,concatenate,Input,UpSampling2D

from tensorflow.keras.models import Model
from tensorflow import metrics
import tensorflow as tf

from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.resnet_v2 import ResNet152V2, ResNet101V2,ResNet50V2
from keras.applications.resnet import ResNet,ResNet50,ResNet101,ResNet152
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.densenet import DenseNet121,DenseNet169,DenseNet201
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.applications.mobilenet import MobileNet
from keras.applications.efficientnet_v2 import EfficientNetV2B0
from keras.applications.efficientnet import EfficientNetB0

In [ ]:
input_size = (512,512,3)
inputs = Input(input_size)

In [ ]:

def  Unet(inputs):
  conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
  conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
  pool1 = MaxPool2D()(conv1)

  conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
  conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
  pool2 = MaxPool2D()(conv2)

  conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
  conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
  conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
  pool3 = MaxPool2D()(conv3)
  conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
  conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
  conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
  drop4 = Dropout(0.5)(conv4)
  pool4 = MaxPool2D()(drop4)

  conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
  conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
  drop5 = Dropout(0.5)(conv5)

  up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
  merge6 = concatenate([drop4,up6], axis = 3)
  conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
  conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

  up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
  merge7 = concatenate([conv3,up7], axis = 3)
  conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
  conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

  up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
  merge8 = concatenate([conv2,up8], axis = 3)
  conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
  conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

  up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
  merge9 = concatenate([conv1,up9], axis = 3)
  conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
  conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
  conv9 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
  conv10 = Conv2D(3, 1, activation = 'softmax')(conv9)

  model = Model(inputs, conv10)
  return model

In [ ]:
# vgg16 = VGG16(include_top=False,input_shape=(512,512,3),pooling=None)
# vgg16.trainable = False
# vgg16_pretrained =  Model(vgg16.input,vgg16.get_layer('block1_conv2').output)

In [ ]:

def  UnetVgg16(inputs):
  vgg16 = VGG16(include_top=False,input_shape=(512,512,3),pooling=None)
  vgg16.trainable = False
  conv1 = vgg16.get_layer('block1_conv1')(inputs)
  conv1 = vgg16.get_layer('block1_conv2')(conv1)
  pool1 = vgg16.get_layer('block1_pool')(conv1)

  conv2 = vgg16.get_layer('block2_conv1')(pool1)
  conv2 = vgg16.get_layer('block2_conv2')(conv2)
  pool2 = vgg16.get_layer('block2_pool')(conv2)

  conv3 = vgg16.get_layer('block3_conv1')(pool2)
  conv3 = vgg16.get_layer('block3_conv2')(conv3)
  conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
  pool3 = vgg16.get_layer('block3_pool')(conv3)
  conv4 = vgg16.get_layer('block4_conv1')(pool3)
  conv4 = vgg16.get_layer('block4_conv2')(conv4)
  conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
  drop4 = Dropout(0.5)(conv4)
  pool4 = vgg16.get_layer('block4_pool')(drop4)

  conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
  conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
  drop5 = Dropout(0.5)(conv5)

  up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
  merge6 = concatenate([drop4,up6], axis = 3)
  conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
  conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

  up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
  merge7 = concatenate([conv3,up7], axis = 3)
  conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
  conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

  up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
  merge8 = concatenate([conv2,up8], axis = 3)
  conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
  conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

  up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
  merge9 = concatenate([conv1,up9], axis = 3)
  conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
  conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
  conv9 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
  conv10 = Conv2D(3, 1, activation = 'softmax')(conv9)

  model = Model(inputs, conv10)
  return model

In [ ]:
# vgg19 = VGG19(include_top=False,input_shape=(512,512,3),pooling=None)
# vgg19.trainable = False
# vgg19_pretrained =  Model(vgg19.input,vgg19.get_layer('block1_conv2').output)

In [ ]:

def  UnetVgg19(inputs):
  vgg19 = VGG19(include_top=False,input_shape=(512,512,3),pooling=None)
  vgg19.trainable = False
  conv1 = vgg19.get_layer('block1_conv1')(inputs)
  conv1 = vgg19.get_layer('block1_conv2')(conv1)
  pool1 = vgg19.get_layer('block1_pool')(conv1)

  conv2 = vgg19.get_layer('block2_conv1')(pool1)
  conv2 = vgg19.get_layer('block2_conv2')(conv2)
  pool2 = vgg19.get_layer('block2_pool')(conv2)

  conv3 = vgg19.get_layer('block3_conv1')(pool2)
  conv3 = vgg19.get_layer('block3_conv2')(conv3)
  conv3 = vgg19.get_layer('block3_conv3')(conv3)
  conv3 = vgg19.get_layer('block3_conv4')(conv3)
  conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
  pool3 = vgg19.get_layer('block3_pool')(conv3)
  conv4 = vgg19.get_layer('block4_conv1')(pool3)
  conv4 = vgg19.get_layer('block4_conv2')(conv4)
  conv4 = vgg19.get_layer('block4_conv3')(conv4)
  conv4 = vgg19.get_layer('block4_conv4')(conv4)
  conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
  drop4 = Dropout(0.5)(conv4)
  pool4 = vgg19.get_layer('block4_pool')(drop4)

  conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
  conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
  drop5 = Dropout(0.5)(conv5)

  up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
  merge6 = concatenate([drop4,up6], axis = 3)
  conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
  conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

  up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
  merge7 = concatenate([conv3,up7], axis = 3)
  conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
  conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

  up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
  merge8 = concatenate([conv2,up8], axis = 3)
  conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
  conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

  up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
  merge9 = concatenate([conv1,up9], axis = 3)
  conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
  conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
  conv9 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
  conv10 = Conv2D(3, 1, activation = 'softmax')(conv9)

  model = Model(inputs, conv10)
  return model

In [ ]:
# resnet152V2 = ResNet152V2(include_top=False,input_shape=(512,512,3),pooling=None)
# resnet152V2.trainable = False
# resnet152V2_pretrained =  Model(resnet152V2.input,resnet152V2.get_layer('conv1_conv').output)

In [ ]:
# resnet101V2 = ResNet101V2(include_top=False,input_shape=(512,512,3),pooling=None)
# resnet101V2.trainable = False
# resnet101V2_pretrained =  Model(resnet101V2.input,resnet101V2.get_layer('conv1_conv').output)

In [ ]:
# resNet50 = ResNet50(include_top=False,input_shape=(512,512,3),pooling=None)
# resNet50.trainable = False
# resNet50_pretrained =  Model(resNet50.input,resNet50.get_layer('conv1_conv').output)

In [ ]:

def  UnetResNet50(inputs):
  resNet50 = ResNet50(include_top=False,input_shape=(512,512,3),pooling=None)
  resNet50.trainable = False
  conv1_pad = resNet50.get_layer('conv1_pad')(inputs)
  conv1_conv = resNet50.get_layer('conv1_conv')(conv1_pad)#concate with 256
  conv1_bn = resNet50.get_layer('conv1_bn')(conv1_conv)
  conv1_relu = resNet50.get_layer('conv1_relu')(conv1_bn)
  pool1_pad = resNet50.get_layer('pool1_pad')(conv1_relu)
  pool1_pool = resNet50.get_layer('pool1_pool')(pool1_pad)
  #---------------------------------------------------------------
  conv2_block1_1_conv = resNet50.get_layer('conv2_block1_1_conv')(pool1_pool)#concate with 128
  conv2_block1_1_bn = resNet50.get_layer('conv2_block1_1_bn')(conv2_block1_1_conv)
  conv2_block1_1_relu = resNet50.get_layer('conv2_block1_1_relu')(conv2_block1_1_bn)
  conv2_block1_2_conv = resNet50.get_layer('conv2_block1_2_conv')(conv2_block1_1_relu)
  conv2_block1_2_bn = resNet50.get_layer('conv2_block1_2_bn')(conv2_block1_2_conv)
  conv2_block1_2_relu = resNet50.get_layer('conv2_block1_2_relu')(conv2_block1_2_bn)

  conv2_block1_0_conv = resNet50.get_layer('conv2_block1_0_conv')(pool1_pool)
  conv2_block1_3_conv = resNet50.get_layer('conv2_block1_3_conv')(conv2_block1_2_relu)
  conv2_block1_0_bn = resNet50.get_layer('conv2_block1_0_bn')(conv2_block1_0_conv)
  conv2_block1_3_bn = resNet50.get_layer('conv2_block1_3_bn')(conv2_block1_3_conv)
  conv2_block1_add = resNet50.get_layer('conv2_block1_add')([conv2_block1_0_bn,conv2_block1_3_bn])
  conv2_block1_out = resNet50.get_layer('conv2_block1_out')(conv2_block1_add)

  conv2_block2_1_conv = resNet50.get_layer('conv2_block2_1_conv')(conv2_block1_out)
  conv2_block2_1_bn = resNet50.get_layer('conv2_block2_1_bn')(conv2_block2_1_conv)
  conv2_block2_1_relu = resNet50.get_layer('conv2_block2_1_relu')(conv2_block2_1_bn)
  conv2_block2_2_conv = resNet50.get_layer('conv2_block2_2_conv')(conv2_block2_1_relu)
  conv2_block2_2_bn = resNet50.get_layer('conv2_block2_2_bn')(conv2_block2_2_conv)
  conv2_block2_2_relu = resNet50.get_layer('conv2_block2_2_relu')(conv2_block2_2_bn)
  conv2_block2_3_conv = resNet50.get_layer('conv2_block2_3_conv')(conv2_block2_2_relu)
  conv2_block2_3_bn = resNet50.get_layer('conv2_block2_3_bn')(conv2_block2_3_conv)
  conv2_block2_add = resNet50.get_layer('conv2_block2_add')([conv2_block1_out,conv2_block2_3_bn])
  conv2_block2_out = resNet50.get_layer('conv2_block2_out')(conv2_block2_add)

  conv2_block3_1_conv = resNet50.get_layer('conv2_block3_1_conv')(conv2_block2_out)
  conv2_block3_1_bn = resNet50.get_layer('conv2_block3_1_bn')(conv2_block3_1_conv)
  conv2_block3_1_relu = resNet50.get_layer('conv2_block3_1_relu')(conv2_block3_1_bn)
  conv2_block3_2_conv = resNet50.get_layer('conv2_block3_2_conv')(conv2_block3_1_relu)
  conv2_block3_2_bn = resNet50.get_layer('conv2_block3_2_bn')(conv2_block3_2_conv)
  conv2_block3_2_relu = resNet50.get_layer('conv2_block3_2_relu')(conv2_block3_2_bn)
  conv2_block3_3_conv = resNet50.get_layer('conv2_block3_3_conv')(conv2_block3_2_relu)
  conv2_block3_3_bn = resNet50.get_layer('conv2_block3_3_bn')(conv2_block3_3_conv)
  conv2_block3_add = resNet50.get_layer('conv2_block3_add')([conv2_block2_out,conv2_block3_3_bn])
  conv2_block3_out = resNet50.get_layer('conv2_block3_out')(conv2_block3_add)
  #-------------------------------------------------------------------------
  conv3_block1_1_conv = resNet50.get_layer('conv3_block1_1_conv')(conv2_block3_out)#concate with 64
  conv3_block1_1_bn = resNet50.get_layer('conv3_block1_1_bn')(conv3_block1_1_conv)
  conv3_block1_1_relu = resNet50.get_layer('conv3_block1_1_relu')(conv3_block1_1_bn)
  conv3_block1_2_conv = resNet50.get_layer('conv3_block1_2_conv')(conv3_block1_1_relu)
  conv3_block1_2_bn = resNet50.get_layer('conv3_block1_2_bn')(conv3_block1_2_conv)
  conv3_block1_2_relu = resNet50.get_layer('conv3_block1_2_relu')(conv3_block1_2_bn)

  conv3_block1_0_conv = resNet50.get_layer('conv3_block1_0_conv')(conv2_block3_out)
  conv3_block1_3_conv = resNet50.get_layer('conv3_block1_3_conv')(conv3_block1_2_relu)
  conv3_block1_0_bn = resNet50.get_layer('conv3_block1_0_bn')(conv3_block1_0_conv)
  conv3_block1_3_bn = resNet50.get_layer('conv3_block1_3_bn')(conv3_block1_3_conv)
  conv3_block1_add = resNet50.get_layer('conv3_block1_add')([conv3_block1_0_bn,conv3_block1_3_bn])
  conv3_block1_out = resNet50.get_layer('conv3_block1_out')(conv3_block1_add)

  conv3_block2_1_conv = resNet50.get_layer('conv3_block2_1_conv')(conv3_block1_out)
  conv3_block2_1_bn = resNet50.get_layer('conv3_block2_1_bn')(conv3_block2_1_conv)
  conv3_block2_1_relu = resNet50.get_layer('conv3_block2_1_relu')(conv3_block2_1_bn)
  conv3_block2_2_conv = resNet50.get_layer('conv3_block2_2_conv')(conv3_block2_1_relu)
  conv3_block2_2_bn = resNet50.get_layer('conv3_block2_2_bn')(conv3_block2_2_conv)
  conv3_block2_2_relu = resNet50.get_layer('conv3_block2_2_relu')(conv3_block2_2_bn)
  conv3_block2_3_conv = resNet50.get_layer('conv3_block2_3_conv')(conv3_block2_2_relu)
  conv3_block2_3_bn = resNet50.get_layer('conv3_block2_3_bn')(conv3_block2_3_conv)
  conv3_block2_add = resNet50.get_layer('conv3_block2_add')([conv3_block1_out,conv3_block2_3_bn])
  conv3_block2_out = resNet50.get_layer('conv3_block2_out')(conv3_block2_add)

  conv3_block3_1_conv = resNet50.get_layer('conv3_block3_1_conv')(conv3_block2_out)
  conv3_block3_1_bn = resNet50.get_layer('conv3_block3_1_bn')(conv3_block3_1_conv)
  conv3_block3_1_relu = resNet50.get_layer('conv3_block3_1_relu')(conv3_block3_1_bn)
  conv3_block3_2_conv = resNet50.get_layer('conv3_block3_2_conv')(conv3_block3_1_relu)
  conv3_block3_2_bn = resNet50.get_layer('conv3_block3_2_bn')(conv3_block3_2_conv)
  conv3_block3_2_relu = resNet50.get_layer('conv3_block3_2_relu')(conv3_block3_2_bn)
  conv3_block3_3_conv = resNet50.get_layer('conv3_block3_3_conv')(conv3_block3_2_relu)
  conv3_block3_3_bn = resNet50.get_layer('conv3_block3_3_bn')(conv3_block3_3_conv)
  conv3_block3_add = resNet50.get_layer('conv3_block3_add')([conv3_block2_out,conv3_block3_3_bn])
  conv3_block3_out = resNet50.get_layer('conv3_block3_out')(conv3_block3_add)

  conv3_block4_1_conv = resNet50.get_layer('conv3_block4_1_conv')(conv3_block3_out)
  conv3_block4_1_bn = resNet50.get_layer('conv3_block4_1_bn')(conv3_block4_1_conv)
  conv3_block4_1_relu = resNet50.get_layer('conv3_block4_1_relu')(conv3_block4_1_bn)
  conv3_block4_2_conv = resNet50.get_layer('conv3_block4_2_conv')(conv3_block4_1_relu)
  conv3_block4_2_bn = resNet50.get_layer('conv3_block4_2_bn')(conv3_block4_2_conv)
  conv3_block4_2_relu = resNet50.get_layer('conv3_block4_2_relu')(conv3_block4_2_bn)
  conv3_block4_3_conv = resNet50.get_layer('conv3_block4_3_conv')(conv3_block4_2_relu)
  conv3_block4_3_bn = resNet50.get_layer('conv3_block4_3_bn')(conv3_block4_3_conv)
  conv3_block4_add = resNet50.get_layer('conv3_block4_add')([conv3_block3_out,conv3_block4_3_bn])
  conv3_block4_out = resNet50.get_layer('conv3_block4_out')(conv3_block4_add)
  #-------------------------------------------------------------------------
  maxpool = MaxPool2D()(conv3_block4_out)

  conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(maxpool)
  conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
  drop5 = Dropout(0.5)(conv5)

  up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
  merge6 = concatenate([conv3_block4_out,up6], axis = 3)
  conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
  conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

  up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
  merge7 = concatenate([conv2_block3_out,up7], axis = 3)
  conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
  conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

  up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
  merge8 = concatenate([conv1_conv,up8], axis = 3)
  conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
  conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

  up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
  # merge9 = concatenate([conv1,up9], axis = 3)
  conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(up9)
  conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

  model = Model(inputs, conv10)
  return model

In [ ]:
# inceptionV3 = InceptionV3(include_top=False,input_shape=(512,512,3),pooling=None)
# inceptionV3.trainable = False
# inceptionV3_pretrained =  Model(inceptionV3.input,inceptionV3.get_layer('block1_conv2').output)

In [ ]:
# x = inceptionV3.layers
# name = []
# for i in x:
#   name.append(i.name)

# y = {v:i for i,v in enumerate(name)}

In [ ]:

def  UnetInceptionV3(inputs):
  inceptionV3 = InceptionV3(include_top=False,input_shape=(512,512,3),pooling=None)
  inceptionV3.trainable = False
  conv2d = inceptionV3.get_layer('conv2d')(inputs)
  batch_normalization = inceptionV3.get_layer('batch_normalization')(conv2d)
  activation = inceptionV3.get_layer('activation')(batch_normalization)
  conv2d_1 = inceptionV3.get_layer('conv2d_1')(activation)
  batch_normalization_1 = inceptionV3.get_layer('conv2d_1')(conv2d_1)
  activation_1 = inceptionV3.get_layer('activation_1')(batch_normalization_1)
  conv2d_2 = inceptionV3.get_layer('conv2d_2')(activation_1)
  batch_normalization_2 = inceptionV3.get_layer('batch_normalization_2')(conv2d_2)
  activation_2 = inceptionV3.get_layer('activation_2')(batch_normalization_2)#concate with 253
  max_pooling2d = inceptionV3.get_layer('max_pooling2d')(activation_2)
  #---------------------------------------------------------------
  conv2d_3 = inceptionV3.get_layer('conv2d_3')(max_pooling2d)
  batch_normalization_3 = inceptionV3.get_layer('batch_normalization_3')(conv2d_3)
  activation_3 = inceptionV3.get_layer('activation_3')(batch_normalization_3)
  conv2d_4 = inceptionV3.get_layer('conv2d_4')(activation_3)
  batch_normalization_4 = inceptionV3.get_layer('batch_normalization_4')(conv2d_4)
  activation_4 = inceptionV3.get_layer('activation_4')(batch_normalization_4)#concate with 124
  max_pooling2d_1 = inceptionV3.get_layer('max_pooling2d_1')(activation_4)
  #---------------------------------------------------------------
  conv2d_8 = inceptionV3.get_layer('conv2d_8')(max_pooling2d_1)
  batch_normalization_8 = inceptionV3.get_layer('batch_normalization_8')(conv2d_8)
  activation_8 = inceptionV3.get_layer('activation_8')(batch_normalization_8)
  conv2d_6 = inceptionV3.get_layer('conv2d_6')(max_pooling2d_1)
  conv2d_9 = inceptionV3.get_layer('conv2d_9')(activation_8)
  batch_normalization_6 = inceptionV3.get_layer('batch_normalization_6')(conv2d_6)
  batch_normalization_9 = inceptionV3.get_layer('batch_normalization_9')(conv2d_9)
  activation_6 = inceptionV3.get_layer('activation_6')(batch_normalization_6)
  activation_9 = inceptionV3.get_layer('activation_9')(batch_normalization_9)
  average_pooling2d = inceptionV3.get_layer('average_pooling2d')(max_pooling2d_1)
  #---------------------------------------------------------------
  conv2d_5 = inceptionV3.get_layer('conv2d_5')(max_pooling2d_1)
  conv2d_7 = inceptionV3.get_layer('conv2d_7')(activation_6)
  conv2d_10 = inceptionV3.get_layer('conv2d_10')(activation_9)
  conv2d_11 = inceptionV3.get_layer('conv2d_11')(average_pooling2d)
  batch_normalization_5 = inceptionV3.get_layer('batch_normalization_5')(conv2d_5)
  batch_normalization_7 = inceptionV3.get_layer('batch_normalization_7')(conv2d_7)
  batch_normalization_10 = inceptionV3.get_layer('batch_normalization_10')(conv2d_10)
  batch_normalization_11 = inceptionV3.get_layer('batch_normalization_11')(conv2d_11)
  activation_5 = inceptionV3.get_layer('activation_5')(batch_normalization_5)
  activation_7 = inceptionV3.get_layer('activation_7')(batch_normalization_7)
  activation_10 = inceptionV3.get_layer('activation_10')(batch_normalization_10)
  activation_11 = inceptionV3.get_layer('activation_11')(batch_normalization_11)
  mixed0 = inceptionV3.get_layer('mixed0')([activation_5,activation_7,activation_10,activation_11])
  conv2d_15 = inceptionV3.get_layer('conv2d_15')(mixed0)
  batch_normalization_15 = inceptionV3.get_layer('batch_normalization_15')(conv2d_15)
  activation_15 = inceptionV3.get_layer('activation_15')(batch_normalization_15)
  conv2d_13 = inceptionV3.get_layer('conv2d_13')(mixed0)
  conv2d_16 = inceptionV3.get_layer('conv2d_16')(activation_15)
  batch_normalization_13 = inceptionV3.get_layer('batch_normalization_13')(conv2d_13)
  batch_normalization_16 = inceptionV3.get_layer('batch_normalization_16')(conv2d_16)
  activation_13 = inceptionV3.get_layer('activation_13')(batch_normalization_13)
  activation_16 = inceptionV3.get_layer('activation_16')(batch_normalization_16)
  average_pooling2d_1 = inceptionV3.get_layer('average_pooling2d_1')(mixed0)
  #-------------------------------------------------------------------------
  conv2d_12 = inceptionV3.get_layer('conv2d_12')(mixed0)
  conv2d_14 = inceptionV3.get_layer('conv2d_14')(activation_13)
  conv2d_17 = inceptionV3.get_layer('conv2d_17')(activation_16)
  conv2d_18 = inceptionV3.get_layer('conv2d_18')(average_pooling2d_1)
  batch_normalization_12 = inceptionV3.get_layer('batch_normalization_12')(conv2d_12)
  batch_normalization_14 = inceptionV3.get_layer('batch_normalization_14')(conv2d_14)
  batch_normalization_17 = inceptionV3.get_layer('batch_normalization_17')(conv2d_17)
  batch_normalization_18 = inceptionV3.get_layer('batch_normalization_18')(conv2d_18)
  activation_12 = inceptionV3.get_layer('activation_12')(batch_normalization_12)
  activation_14 = inceptionV3.get_layer('activation_14')(batch_normalization_14)
  activation_17 = inceptionV3.get_layer('activation_17')(batch_normalization_17)
  activation_18 = inceptionV3.get_layer('activation_18')(batch_normalization_18)
  mixed1 = inceptionV3.get_layer('mixed1')([activation_12,activation_14,activation_17,activation_18])
  conv2d_22 = inceptionV3.get_layer('conv2d_22')(mixed1)
  batch_normalization_22 = inceptionV3.get_layer('batch_normalization_22')(conv2d_22)
  activation_22 = inceptionV3.get_layer('activation_22')(batch_normalization_22)
  conv2d_20 = inceptionV3.get_layer('conv2d_20')(mixed1)
  conv2d_23 = inceptionV3.get_layer('conv2d_23')(activation_22)
  batch_normalization_20 = inceptionV3.get_layer('batch_normalization_20')(conv2d_20)
  batch_normalization_23 = inceptionV3.get_layer('batch_normalization_23')(conv2d_23)
  activation_20 = inceptionV3.get_layer('activation_20')(batch_normalization_20)
  activation_23 = inceptionV3.get_layer('activation_23')(batch_normalization_23)
  average_pooling2d_2 = inceptionV3.get_layer('average_pooling2d_2')(mixed1)
  #-------------------------------------------------------------------------
  conv2d_19 = inceptionV3.get_layer('conv2d_19')(mixed1)
  conv2d_21 = inceptionV3.get_layer('conv2d_21')(activation_20)
  conv2d_24 = inceptionV3.get_layer('conv2d_24')(activation_23)
  conv2d_25 = inceptionV3.get_layer('conv2d_25')(average_pooling2d_2)
  batch_normalization_19 = inceptionV3.get_layer('batch_normalization_19')(conv2d_19)
  batch_normalization_21 = inceptionV3.get_layer('batch_normalization_21')(conv2d_21)
  batch_normalization_24 = inceptionV3.get_layer('batch_normalization_24')(conv2d_24)
  batch_normalization_25 = inceptionV3.get_layer('batch_normalization_25')(conv2d_25)
  activation_19 = inceptionV3.get_layer('activation_19')(batch_normalization_19)
  activation_21 = inceptionV3.get_layer('activation_21')(batch_normalization_21)
  activation_24 = inceptionV3.get_layer('activation_24')(batch_normalization_24)
  activation_25 = inceptionV3.get_layer('activation_25')(batch_normalization_25)
  mixed2 = inceptionV3.get_layer('mixed2')([activation_19,activation_21,activation_24,activation_25])#concate with 61
  # conv2d_27 = inceptionV3.get_layer('conv2d_27')(mixed2)
  # batch_normalization_27 = inceptionV3.get_layer('batch_normalization_27')(conv2d_27)
  # activation_27 = inceptionV3.get_layer('activation_27')(batch_normalization_27)
  # conv2d_28 = inceptionV3.get_layer('conv2d_28')(activation_27)
  # batch_normalization_28 = inceptionV3.get_layer('batch_normalization_28')(conv2d_28)
  # activation_28 = inceptionV3.get_layer('activation_28')(batch_normalization_28)
  # conv2d_26 = inceptionV3.get_layer('conv2d_26')(mixed2)
  # conv2d_29 = inceptionV3.get_layer('conv2d_29')(activation_28)
  # batch_normalization_26 = inceptionV3.get_layer('batch_normalization_26')(conv2d_26)
  # batch_normalization_29 = inceptionV3.get_layer('batch_normalization_29')(conv2d_29)
  # activation_26 = inceptionV3.get_layer('activation_26')(batch_normalization_26)
  # activation_29 = inceptionV3.get_layer('activation_29')(batch_normalization_29)
  # max_pooling2d_2 = inceptionV3.get_layer('average_pooling2d_2')(mixed2)
  #-------------------------------------------------------------------------
  # maxpool = MaxPool2D()(mixed2)

  # conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(maxpool)
  # conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
  # drop5 = Dropout(0.5)(conv5)
  up6 = Conv2D(192, (2,2), activation = 'relu', padding = 'valid', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(mixed2))
  up6 = tf.keras.layers.ZeroPadding2D(padding=1)(up6)
  up6 = tf.keras.layers.ZeroPadding2D(padding=1)(up6)

  merge6 = concatenate([activation_3,up6], axis = 3)
  conv6 = Conv2D(192, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
  conv6 = Conv2D(192, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

  up7 = Conv2D(64, 2, activation = 'relu', padding = 'valid', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
  up7 = tf.keras.layers.ZeroPadding2D(padding=1)(up7)
  up7 = tf.keras.layers.ZeroPadding2D(padding=1)(up7)
  up7 = tf.keras.layers.ZeroPadding2D(padding=1)(up7)
  merge7 = concatenate([conv2d,up7], axis = 3)
  conv7 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
  conv7 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

  up8 = Conv2D(24, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))

  up8 = tf.keras.layers.ZeroPadding2D(padding=1)(up8)
  # merge8 = concatenate([activation_2,up8], axis = 3)
  # conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
  # conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

  # up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
  # # merge9 = concatenate([conv1,up9], axis = 3)
  # conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(up9)
  conv10 = Conv2D(1, 1, activation = 'sigmoid')(up8)

  model = Model(inputs, conv10)
  return model

In [ ]:
# inceptionResNetV2 = InceptionResNetV2(include_top=False,input_shape=(512,512,3),pooling=None)
# inceptionResNetV2.trainable = False
# inceptionResNetV2_pretrained =  Model(inceptionResNetV2.input,inceptionResNetV2.get_layer('block1_conv2').output)

219070464/219055592 [==============================] - 2s 0us/step


In [ ]:

def  UnetInceptionResNetV2(inputs):
  inceptionResNetV2 = InceptionResNetV2(include_top=False,input_shape=(512,512,3),pooling=None)
  inceptionResNetV2.trainable = False
  conv2d = inceptionResNetV2.get_layer('conv2d')(inputs)
  batch_normalization = inceptionResNetV2.get_layer('batch_normalization')(conv2d)
  activation = inceptionResNetV2.get_layer('activation')(batch_normalization)
  conv2d_1 = inceptionResNetV2.get_layer('conv2d_1')(activation)
  batch_normalization_1 = inceptionResNetV2.get_layer('conv2d_1')(conv2d_1)
  activation_1 = inceptionResNetV2.get_layer('activation_1')(batch_normalization_1)
  conv2d_2 = inceptionResNetV2.get_layer('conv2d_2')(activation_1)
  batch_normalization_2 = inceptionResNetV2.get_layer('batch_normalization_2')(conv2d_2)
  activation_2 = inceptionResNetV2.get_layer('activation_2')(batch_normalization_2)#concate with 253
  max_pooling2d = inceptionResNetV2.get_layer('max_pooling2d')(activation_2)
  #---------------------------------------------------------------
  conv2d_3 = inceptionResNetV2.get_layer('conv2d_3')(max_pooling2d)
  batch_normalization_3 = inceptionResNetV2.get_layer('batch_normalization_3')(conv2d_3)
  activation_3 = inceptionResNetV2.get_layer('activation_3')(batch_normalization_3)
  conv2d_4 = inceptionResNetV2.get_layer('conv2d_4')(activation_3)
  batch_normalization_4 = inceptionResNetV2.get_layer('batch_normalization_4')(conv2d_4)
  activation_4 = inceptionResNetV2.get_layer('activation_4')(batch_normalization_4)#concate with 124
  max_pooling2d_1 = inceptionResNetV2.get_layer('max_pooling2d_1')(activation_4)
  #---------------------------------------------------------------
  conv2d_8 = inceptionResNetV2.get_layer('conv2d_8')(max_pooling2d_1)
  batch_normalization_8 = inceptionResNetV2.get_layer('batch_normalization_8')(conv2d_8)
  activation_8 = inceptionResNetV2.get_layer('activation_8')(batch_normalization_8)
  conv2d_6 = inceptionResNetV2.get_layer('conv2d_6')(max_pooling2d_1)
  conv2d_9 = inceptionResNetV2.get_layer('conv2d_9')(activation_8)
  batch_normalization_6 = inceptionResNetV2.get_layer('batch_normalization_6')(conv2d_6)
  batch_normalization_9 = inceptionResNetV2.get_layer('batch_normalization_9')(conv2d_9)
  activation_6 = inceptionResNetV2.get_layer('activation_6')(batch_normalization_6)
  activation_9 = inceptionResNetV2.get_layer('activation_9')(batch_normalization_9)
  average_pooling2d = inceptionResNetV2.get_layer('average_pooling2d')(max_pooling2d_1)
  #---------------------------------------------------------------
  conv2d_5 = inceptionResNetV2.get_layer('conv2d_5')(max_pooling2d_1)
  conv2d_7 = inceptionResNetV2.get_layer('conv2d_7')(activation_6)
  conv2d_10 = inceptionResNetV2.get_layer('conv2d_10')(activation_9)
  conv2d_11 = inceptionResNetV2.get_layer('conv2d_11')(average_pooling2d)
  batch_normalization_5 = inceptionResNetV2.get_layer('batch_normalization_5')(conv2d_5)
  batch_normalization_7 = inceptionResNetV2.get_layer('batch_normalization_7')(conv2d_7)
  batch_normalization_10 = inceptionResNetV2.get_layer('batch_normalization_10')(conv2d_10)
  batch_normalization_11 = inceptionResNetV2.get_layer('batch_normalization_11')(conv2d_11)
  activation_5 = inceptionResNetV2.get_layer('activation_5')(batch_normalization_5)
  activation_7 = inceptionResNetV2.get_layer('activation_7')(batch_normalization_7)
  activation_10 = inceptionResNetV2.get_layer('activation_10')(batch_normalization_10)
  activation_11 = inceptionResNetV2.get_layer('activation_11')(batch_normalization_11)
  mixed_5b = inceptionResNetV2.get_layer('mixed_5b')([activation_5,activation_7,activation_10,activation_11])
  #-------------------------------------------------------------------------
  conv2d_15 = inceptionResNetV2.get_layer('conv2d_15')(mixed_5b)
  batch_normalization_15 = inceptionResNetV2.get_layer('batch_normalization_15')(conv2d_15)
  activation_15 = inceptionResNetV2.get_layer('activation_15')(batch_normalization_15)
  conv2d_13 = inceptionResNetV2.get_layer('conv2d_13')(mixed_5b)
  conv2d_16 = inceptionResNetV2.get_layer('conv2d_16')(activation_15)
  batch_normalization_13 = inceptionResNetV2.get_layer('batch_normalization_13')(conv2d_13)
  batch_normalization_16 = inceptionResNetV2.get_layer('batch_normalization_16')(conv2d_16)
  activation_13 = inceptionResNetV2.get_layer('activation_13')(batch_normalization_13)
  activation_16 = inceptionResNetV2.get_layer('activation_16')(batch_normalization_16)
  conv2d_12 = inceptionResNetV2.get_layer('conv2d_12')(mixed_5b)
  conv2d_14 = inceptionResNetV2.get_layer('conv2d_14')(activation_13)
  conv2d_17 = inceptionResNetV2.get_layer('conv2d_17')(activation_16)
  batch_normalization_12 = inceptionResNetV2.get_layer('batch_normalization_12')(conv2d_12)
  batch_normalization_14 = inceptionResNetV2.get_layer('batch_normalization_14')(conv2d_14)
  batch_normalization_17 = inceptionResNetV2.get_layer('batch_normalization_17')(conv2d_17)
  activation_12 = inceptionResNetV2.get_layer('activation_12')(batch_normalization_12)
  activation_14 = inceptionResNetV2.get_layer('activation_14')(batch_normalization_14)
  activation_17 = inceptionResNetV2.get_layer('activation_17')(batch_normalization_17)
  block35_1_mixed = inceptionResNetV2.get_layer('block35_1_mixed')([activation_12,activation_14,activation_17])
  block35_1_conv = inceptionResNetV2.get_layer('block35_1_conv')(block35_1_mixed)
  block35_1 = inceptionResNetV2.get_layer('block35_1')([mixed_5b,block35_1_conv])
  block35_1_ac = inceptionResNetV2.get_layer('block35_1_ac')(block35_1)
  #-------------------------------------------------------------------------
  conv2d_21 = inceptionResNetV2.get_layer('conv2d_21')(block35_1_ac)
  batch_normalization_21 = inceptionResNetV2.get_layer('batch_normalization_21')(conv2d_21)
  activation_21 = inceptionResNetV2.get_layer('activation_21')(batch_normalization_21)
  conv2d_19 = inceptionResNetV2.get_layer('conv2d_19')(block35_1_ac)
  conv2d_22 = inceptionResNetV2.get_layer('conv2d_22')(activation_21)
  batch_normalization_19 = inceptionResNetV2.get_layer('batch_normalization_19')(conv2d_19)
  batch_normalization_22 = inceptionResNetV2.get_layer('batch_normalization_22')(conv2d_22)
  activation_19 = inceptionResNetV2.get_layer('activation_19')(batch_normalization_19)
  activation_22 = inceptionResNetV2.get_layer('activation_22')(batch_normalization_22)
  conv2d_18 = inceptionResNetV2.get_layer('conv2d_18')(block35_1_ac)
  conv2d_20 = inceptionResNetV2.get_layer('conv2d_20')(activation_19)
  conv2d_23 = inceptionResNetV2.get_layer('conv2d_23')(activation_22)
  batch_normalization_18 = inceptionResNetV2.get_layer('batch_normalization_18')(conv2d_18)
  batch_normalization_20 = inceptionResNetV2.get_layer('batch_normalization_20')(conv2d_20)
  batch_normalization_23 = inceptionResNetV2.get_layer('batch_normalization_23')(conv2d_23)
  activation_18 = inceptionResNetV2.get_layer('activation_18')(batch_normalization_18)
  activation_20 = inceptionResNetV2.get_layer('activation_20')(batch_normalization_20)
  activation_23 = inceptionResNetV2.get_layer('activation_23')(batch_normalization_23)
  block35_2_mixed = inceptionResNetV2.get_layer('block35_2_mixed')([activation_18,activation_20,activation_23])
  block35_2_conv = inceptionResNetV2.get_layer('block35_2_conv')(block35_2_mixed)
  block35_2 = inceptionResNetV2.get_layer('block35_2')([block35_1_ac,block35_2_conv])
  block35_2_ac = inceptionResNetV2.get_layer('block35_2_ac')(block35_2)
  #-------------------------------------------------------------------------
  conv2d_27 = inceptionResNetV2.get_layer('conv2d_27')(block35_2_ac)
  batch_normalization_27 = inceptionResNetV2.get_layer('batch_normalization_27')(conv2d_27)
  activation_27 = inceptionResNetV2.get_layer('activation_27')(batch_normalization_27)
  conv2d_25 = inceptionResNetV2.get_layer('conv2d_25')(block35_2_ac)
  conv2d_28 = inceptionResNetV2.get_layer('conv2d_28')(activation_27)
  batch_normalization_25 = inceptionResNetV2.get_layer('batch_normalization_25')(conv2d_25)
  batch_normalization_28 = inceptionResNetV2.get_layer('batch_normalization_28')(conv2d_28)
  activation_25 = inceptionResNetV2.get_layer('activation_25')(batch_normalization_25)
  activation_28 = inceptionResNetV2.get_layer('activation_28')(batch_normalization_28)
  conv2d_24 = inceptionResNetV2.get_layer('conv2d_24')(block35_2_ac)
  conv2d_26 = inceptionResNetV2.get_layer('conv2d_26')(activation_25)
  conv2d_29 = inceptionResNetV2.get_layer('conv2d_29')(activation_28)
  batch_normalization_24 = inceptionResNetV2.get_layer('batch_normalization_24')(conv2d_24)
  batch_normalization_26 = inceptionResNetV2.get_layer('batch_normalization_26')(conv2d_26)
  batch_normalization_29 = inceptionResNetV2.get_layer('batch_normalization_29')(conv2d_29)
  activation_24 = inceptionResNetV2.get_layer('activation_24')(batch_normalization_24)
  activation_26 = inceptionResNetV2.get_layer('activation_26')(batch_normalization_26)
  activation_29 = inceptionResNetV2.get_layer('activation_29')(batch_normalization_29)
  block35_3_mixed = inceptionResNetV2.get_layer('block35_3_mixed')([activation_24,activation_26,activation_29])
  block35_3_conv = inceptionResNetV2.get_layer('block35_3_conv')(block35_3_mixed)
  block35_3 = inceptionResNetV2.get_layer('block35_3')([block35_2_ac,block35_3_conv])
  block35_3_ac = inceptionResNetV2.get_layer('block35_3_ac')(block35_3)
  #-------------------------------------------------------------------------
  conv2d_33 = inceptionResNetV2.get_layer('conv2d_33')(block35_3_ac)
  batch_normalization_33 = inceptionResNetV2.get_layer('batch_normalization_33')(conv2d_33)
  activation_33 = inceptionResNetV2.get_layer('activation_33')(batch_normalization_33)
  conv2d_31 = inceptionResNetV2.get_layer('conv2d_31')(block35_3_ac)
  conv2d_34 = inceptionResNetV2.get_layer('conv2d_34')(activation_33)
  batch_normalization_31 = inceptionResNetV2.get_layer('batch_normalization_31')(conv2d_31)
  batch_normalization_34 = inceptionResNetV2.get_layer('batch_normalization_34')(conv2d_34)
  activation_31 = inceptionResNetV2.get_layer('activation_31')(batch_normalization_31)
  activation_34 = inceptionResNetV2.get_layer('activation_34')(batch_normalization_34)
  conv2d_30 = inceptionResNetV2.get_layer('conv2d_30')(block35_3_ac)
  conv2d_32 = inceptionResNetV2.get_layer('conv2d_32')(activation_31)
  conv2d_35 = inceptionResNetV2.get_layer('conv2d_35')(activation_34)
  batch_normalization_30 = inceptionResNetV2.get_layer('batch_normalization_30')(conv2d_30)
  batch_normalization_32 = inceptionResNetV2.get_layer('batch_normalization_32')(conv2d_32)
  batch_normalization_35 = inceptionResNetV2.get_layer('batch_normalization_35')(conv2d_35)
  activation_30 = inceptionResNetV2.get_layer('activation_30')(batch_normalization_30)
  activation_32 = inceptionResNetV2.get_layer('activation_32')(batch_normalization_32)
  activation_35 = inceptionResNetV2.get_layer('activation_35')(batch_normalization_35)
  block35_4_mixed = inceptionResNetV2.get_layer('block35_4_mixed')([activation_30,activation_32,activation_35])
  block35_4_conv = inceptionResNetV2.get_layer('block35_4_conv')(block35_4_mixed)
  block35_4 = inceptionResNetV2.get_layer('block35_4')([block35_3_ac,block35_4_conv])
  block35_4_ac = inceptionResNetV2.get_layer('block35_4_ac')(block35_4)
  #-------------------------------------------------------------------------
  conv2d_39 = inceptionResNetV2.get_layer('conv2d_39')(block35_4_ac)
  batch_normalization_39 = inceptionResNetV2.get_layer('batch_normalization_39')(conv2d_39)
  activation_39 = inceptionResNetV2.get_layer('activation_39')(batch_normalization_39)
  conv2d_37 = inceptionResNetV2.get_layer('conv2d_37')(block35_4_ac)
  conv2d_40 = inceptionResNetV2.get_layer('conv2d_40')(activation_39)
  batch_normalization_37 = inceptionResNetV2.get_layer('batch_normalization_37')(conv2d_37)
  batch_normalization_40 = inceptionResNetV2.get_layer('batch_normalization_40')(conv2d_40)
  activation_37 = inceptionResNetV2.get_layer('activation_37')(batch_normalization_37)
  activation_40 = inceptionResNetV2.get_layer('activation_40')(batch_normalization_40)
  conv2d_36 = inceptionResNetV2.get_layer('conv2d_36')(block35_4_ac)
  conv2d_38 = inceptionResNetV2.get_layer('conv2d_38')(activation_37)
  conv2d_41 = inceptionResNetV2.get_layer('conv2d_41')(activation_40)
  batch_normalization_36 = inceptionResNetV2.get_layer('batch_normalization_36')(conv2d_36)
  batch_normalization_38 = inceptionResNetV2.get_layer('batch_normalization_38')(conv2d_38)
  batch_normalization_41 = inceptionResNetV2.get_layer('batch_normalization_41')(conv2d_41)
  activation_36 = inceptionResNetV2.get_layer('activation_36')(batch_normalization_36)
  activation_38 = inceptionResNetV2.get_layer('activation_38')(batch_normalization_38)
  activation_41 = inceptionResNetV2.get_layer('activation_41')(batch_normalization_41)
  block35_5_mixed = inceptionResNetV2.get_layer('block35_5_mixed')([activation_36,activation_38,activation_41])
  block35_5_conv = inceptionResNetV2.get_layer('block35_5_conv')(block35_5_mixed)
  block35_5 = inceptionResNetV2.get_layer('block35_5')([block35_4_ac,block35_5_conv])
  block35_5_ac = inceptionResNetV2.get_layer('block35_5_ac')(block35_5)
  #-------------------------------------------------------------------------
  conv2d_45 = inceptionResNetV2.get_layer('conv2d_45')(block35_5_ac)
  batch_normalization_45= inceptionResNetV2.get_layer('batch_normalization_45')(conv2d_45)
  activation_45 = inceptionResNetV2.get_layer('activation_45')(batch_normalization_45)
  conv2d_43 = inceptionResNetV2.get_layer('conv2d_43')(block35_5_ac)
  conv2d_46 = inceptionResNetV2.get_layer('conv2d_46')(activation_45)
  batch_normalization_43 = inceptionResNetV2.get_layer('batch_normalization_43')(conv2d_43)
  batch_normalization_46 = inceptionResNetV2.get_layer('batch_normalization_46')(conv2d_46)
  activation_43 = inceptionResNetV2.get_layer('activation_43')(batch_normalization_43)
  activation_46 = inceptionResNetV2.get_layer('activation_46')(batch_normalization_46)
  conv2d_42 = inceptionResNetV2.get_layer('conv2d_42')(block35_5_ac)
  conv2d_44 = inceptionResNetV2.get_layer('conv2d_44')(activation_43)
  conv2d_47 = inceptionResNetV2.get_layer('conv2d_47')(activation_46)
  batch_normalization_42 = inceptionResNetV2.get_layer('batch_normalization_42')(conv2d_42)
  batch_normalization_44 = inceptionResNetV2.get_layer('batch_normalization_44')(conv2d_44)
  batch_normalization_47 = inceptionResNetV2.get_layer('batch_normalization_47')(conv2d_47)
  activation_42 = inceptionResNetV2.get_layer('activation_42')(batch_normalization_42)
  activation_44 = inceptionResNetV2.get_layer('activation_44')(batch_normalization_44)
  activation_47 = inceptionResNetV2.get_layer('activation_47')(batch_normalization_47)
  block35_6_mixed = inceptionResNetV2.get_layer('block35_6_mixed')([activation_42,activation_44,activation_47])
  block35_6_conv = inceptionResNetV2.get_layer('block35_6_conv')(block35_6_mixed)
  block35_6 = inceptionResNetV2.get_layer('block35_6')([block35_5_ac,block35_6_conv])
  block35_6_ac = inceptionResNetV2.get_layer('block35_6_ac')(block35_6)
  #-------------------------------------------------------------------------
  conv2d_51 = inceptionResNetV2.get_layer('conv2d_51')(block35_6_ac)
  batch_normalization_51 = inceptionResNetV2.get_layer('batch_normalization_51')(conv2d_51)
  activation_51 = inceptionResNetV2.get_layer('activation_51')(batch_normalization_51)
  conv2d_49 = inceptionResNetV2.get_layer('conv2d_49')(block35_6_ac)
  conv2d_52 = inceptionResNetV2.get_layer('conv2d_52')(activation_51)
  batch_normalization_49 = inceptionResNetV2.get_layer('batch_normalization_49')(conv2d_49)
  batch_normalization_52 = inceptionResNetV2.get_layer('batch_normalization_52')(conv2d_52)
  activation_49 = inceptionResNetV2.get_layer('activation_49')(batch_normalization_49)
  activation_52 = inceptionResNetV2.get_layer('activation_52')(batch_normalization_52)
  conv2d_48 = inceptionResNetV2.get_layer('conv2d_48')(block35_6_ac)
  conv2d_50 = inceptionResNetV2.get_layer('conv2d_50')(activation_49)
  conv2d_53 = inceptionResNetV2.get_layer('conv2d_53')(activation_52)
  batch_normalization_48 = inceptionResNetV2.get_layer('batch_normalization_48')(conv2d_48)
  batch_normalization_50 = inceptionResNetV2.get_layer('batch_normalization_50')(conv2d_50)
  batch_normalization_53 = inceptionResNetV2.get_layer('batch_normalization_53')(conv2d_53)
  activation_48 = inceptionResNetV2.get_layer('activation_48')(batch_normalization_48)
  activation_50 = inceptionResNetV2.get_layer('activation_50')(batch_normalization_50)
  activation_53 = inceptionResNetV2.get_layer('activation_53')(batch_normalization_53)
  block35_7_mixed = inceptionResNetV2.get_layer('block35_7_mixed')([activation_48,activation_50,activation_53])
  block35_7_conv = inceptionResNetV2.get_layer('block35_7_conv')(block35_7_mixed)
  block35_7 = inceptionResNetV2.get_layer('block35_7')([block35_6_ac,block35_7_conv])
  block35_7_ac = inceptionResNetV2.get_layer('block35_7_ac')(block35_7)
  #-------------------------------------------------------------------------
  conv2d_57 = inceptionResNetV2.get_layer('conv2d_57')(block35_7_ac)
  batch_normalization_57 = inceptionResNetV2.get_layer('batch_normalization_57')(conv2d_57)
  activation_57 = inceptionResNetV2.get_layer('activation_57')(batch_normalization_57)
  conv2d_55 = inceptionResNetV2.get_layer('conv2d_55')(block35_7_ac)
  conv2d_58 = inceptionResNetV2.get_layer('conv2d_58')(activation_57)
  batch_normalization_55 = inceptionResNetV2.get_layer('batch_normalization_55')(conv2d_55)
  batch_normalization_58 = inceptionResNetV2.get_layer('batch_normalization_58')(conv2d_58)
  activation_55 = inceptionResNetV2.get_layer('activation_55')(batch_normalization_55)
  activation_58 = inceptionResNetV2.get_layer('activation_58')(batch_normalization_58)
  conv2d_54 = inceptionResNetV2.get_layer('conv2d_54')(block35_7_ac)
  conv2d_56 = inceptionResNetV2.get_layer('conv2d_56')(activation_55)
  conv2d_59 = inceptionResNetV2.get_layer('conv2d_59')(activation_58)
  batch_normalization_54 = inceptionResNetV2.get_layer('batch_normalization_54')(conv2d_54)
  batch_normalization_56 = inceptionResNetV2.get_layer('batch_normalization_56')(conv2d_56)
  batch_normalization_59 = inceptionResNetV2.get_layer('batch_normalization_59')(conv2d_59)
  activation_54 = inceptionResNetV2.get_layer('activation_54')(batch_normalization_54)
  activation_56 = inceptionResNetV2.get_layer('activation_56')(batch_normalization_56)
  activation_59 = inceptionResNetV2.get_layer('activation_59')(batch_normalization_59)
  block35_8_mixed = inceptionResNetV2.get_layer('block35_8_mixed')([activation_54,activation_56,activation_59])
  block35_8_conv = inceptionResNetV2.get_layer('block35_8_conv')(block35_8_mixed)
  block35_8 = inceptionResNetV2.get_layer('block35_8')([block35_7_ac,block35_8_conv])
  block35_8_ac = inceptionResNetV2.get_layer('block35_8_ac')(block35_8)
  #-------------------------------------------------------------------------
  conv2d_63 = inceptionResNetV2.get_layer('conv2d_63')(block35_8_ac)
  batch_normalization_63 = inceptionResNetV2.get_layer('batch_normalization_63')(conv2d_63)
  activation_63 = inceptionResNetV2.get_layer('activation_63')(batch_normalization_63)
  conv2d_61 = inceptionResNetV2.get_layer('conv2d_61')(block35_8_ac)
  conv2d_64 = inceptionResNetV2.get_layer('conv2d_64')(activation_63)
  batch_normalization_61 = inceptionResNetV2.get_layer('batch_normalization_61')(conv2d_61)
  batch_normalization_64 = inceptionResNetV2.get_layer('batch_normalization_64')(conv2d_64)
  activation_61 = inceptionResNetV2.get_layer('activation_61')(batch_normalization_61)
  activation_64 = inceptionResNetV2.get_layer('activation_64')(batch_normalization_64)
  conv2d_60 = inceptionResNetV2.get_layer('conv2d_60')(block35_8_ac)
  conv2d_62 = inceptionResNetV2.get_layer('conv2d_62')(activation_61)
  conv2d_65 = inceptionResNetV2.get_layer('conv2d_65')(activation_64)
  batch_normalization_60 = inceptionResNetV2.get_layer('batch_normalization_60')(conv2d_60)
  batch_normalization_62 = inceptionResNetV2.get_layer('batch_normalization_62')(conv2d_62)
  batch_normalization_65 = inceptionResNetV2.get_layer('batch_normalization_65')(conv2d_65)
  activation_60 = inceptionResNetV2.get_layer('activation_60')(batch_normalization_60)
  activation_62 = inceptionResNetV2.get_layer('activation_62')(batch_normalization_62)
  activation_65 = inceptionResNetV2.get_layer('activation_65')(batch_normalization_65)
  block35_9_mixed = inceptionResNetV2.get_layer('block35_9_mixed')([activation_60,activation_62,activation_65])
  block35_9_conv = inceptionResNetV2.get_layer('block35_9_conv')(block35_9_mixed)
  block35_9 = inceptionResNetV2.get_layer('block35_9')([block35_8_ac,block35_9_conv])
  block35_9_ac = inceptionResNetV2.get_layer('block35_9_ac')(block35_9)
  #-------------------------------------------------------------------------
  conv2d_69 = inceptionResNetV2.get_layer('conv2d_69')(block35_9_ac)
  batch_normalization_69 = inceptionResNetV2.get_layer('batch_normalization_69')(conv2d_69)
  activation_69 = inceptionResNetV2.get_layer('activation_69')(batch_normalization_69)
  conv2d_67 = inceptionResNetV2.get_layer('conv2d_67')(block35_9_ac)
  conv2d_70 = inceptionResNetV2.get_layer('conv2d_70')(activation_69)
  batch_normalization_67 = inceptionResNetV2.get_layer('batch_normalization_67')(conv2d_67)
  batch_normalization_70 = inceptionResNetV2.get_layer('batch_normalization_70')(conv2d_70)
  activation_67 = inceptionResNetV2.get_layer('activation_67')(batch_normalization_67)
  activation_70 = inceptionResNetV2.get_layer('activation_70')(batch_normalization_70)
  conv2d_66 = inceptionResNetV2.get_layer('conv2d_66')(block35_9_ac)
  conv2d_68 = inceptionResNetV2.get_layer('conv2d_68')(activation_67)
  conv2d_71 = inceptionResNetV2.get_layer('conv2d_71')(activation_70)
  batch_normalization_66 = inceptionResNetV2.get_layer('batch_normalization_66')(conv2d_66)
  batch_normalization_68 = inceptionResNetV2.get_layer('batch_normalization_68')(conv2d_68)
  batch_normalization_71 = inceptionResNetV2.get_layer('batch_normalization_71')(conv2d_71)
  activation_66 = inceptionResNetV2.get_layer('activation_66')(batch_normalization_66)
  activation_68 = inceptionResNetV2.get_layer('activation_68')(batch_normalization_68)
  activation_71 = inceptionResNetV2.get_layer('activation_71')(batch_normalization_71)
  block35_10_mixed = inceptionResNetV2.get_layer('block35_10_mixed')([activation_66,activation_68,activation_71])
  block35_10_conv = inceptionResNetV2.get_layer('block35_10_conv')(block35_10_mixed)
  block35_10 = inceptionResNetV2.get_layer('block35_10')([block35_9_ac,block35_10_conv])
  block35_10_ac = inceptionResNetV2.get_layer('block35_10_ac')(block35_10)
  #-------------------------------------------------------------------------
  conv2d_73 = inceptionResNetV2.get_layer('conv2d_73')(block35_10_ac)
  batch_normalization_73 = inceptionResNetV2.get_layer('batch_normalization_73')(conv2d_73)
  activation_73 = inceptionResNetV2.get_layer('activation_73')(batch_normalization_73)
  conv2d_74 = inceptionResNetV2.get_layer('conv2d_74')(activation_73)
  batch_normalization_74 = inceptionResNetV2.get_layer('batch_normalization_74')(conv2d_74)
  activation_74 = inceptionResNetV2.get_layer('activation_74')(batch_normalization_74)
  #-------------------------------------------------------------------------
  # maxpool = MaxPool2D()(mixed2)

  # conv5 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(maxpool)
  # conv5 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
  # drop5 = Dropout(0.5)(conv5)

  up6 = Conv2D(192, (2,2), activation = 'relu', padding = 'valid', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(activation_74))
  up6 = tf.keras.layers.ZeroPadding2D(padding=1)(up6)
  up6 = tf.keras.layers.ZeroPadding2D(padding=1)(up6)

  merge6 = concatenate([activation_3,up6], axis = 3)
  conv6 = Conv2D(192, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
  conv6 = Conv2D(192, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

  up7 = Conv2D(64, 2, activation = 'relu', padding = 'valid', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
  up7 = tf.keras.layers.ZeroPadding2D(padding=1)(up7)
  up7 = tf.keras.layers.ZeroPadding2D(padding=1)(up7)
  up7 = tf.keras.layers.ZeroPadding2D(padding=1)(up7)
  merge7 = concatenate([conv2d,up7], axis = 3)
  conv7 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
  conv7 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

  up8 = Conv2D(24, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))

  up8 = tf.keras.layers.ZeroPadding2D(padding=1)(up8)
  # merge8 = concatenate([activation_2,up8], axis = 3)
  # conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
  # conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

  # up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
  # # merge9 = concatenate([conv1,up9], axis = 3)
  # conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(up9)
  conv10 = Conv2D(1, 1, activation = 'sigmoid')(up8)

  model = Model(inputs, conv10)
  return model

In [ ]:
# denseNet121 = DenseNet121(include_top=False,input_shape=(512,512,3),pooling=None)
# denseNet121.trainable = False
# denseNet121_pretrained =  Model(denseNet121.input,denseNet121.get_layer('block1_conv2').output)

29097984/29084464 [==============================] - 0s 0us/step


In [ ]:
input_size = (512,512,3)
inputs = Input(input_size)
def  UnetDenseNet121(inputs):
  denseNet121 = DenseNet121(include_top=False,input_shape=(512,512,3),pooling=None)
  denseNet121.trainable = False
  zero_padding2d = denseNet121.get_layer('zero_padding2d')(inputs)
  conv1conv = denseNet121.get_layer('conv1/conv')(zero_padding2d)#concate with 256
  conv1bn = denseNet121.get_layer('conv1/bn')(conv1conv)
  conv1relu = denseNet121.get_layer('conv1/relu')(conv1bn)
  zero_padding2d_1 = denseNet121.get_layer('zero_padding2d_1')(conv1relu)
  pool1 = denseNet121.get_layer('pool1')(zero_padding2d_1)
  #---------------------------------------------------------------
  conv2_block1_0_bn = denseNet121.get_layer('conv2_block1_0_bn')(pool1)#concate with 128
  conv2_block1_0_relu = denseNet121.get_layer('conv2_block1_0_relu')(conv2_block1_0_bn)
  conv2_block1_1_conv = denseNet121.get_layer('conv2_block1_1_conv')(conv2_block1_0_relu)
  conv2_block1_1_bn = denseNet121.get_layer('conv2_block1_1_bn')(conv2_block1_1_conv)
  conv2_block1_1_relu = denseNet121.get_layer('conv2_block1_1_relu')(conv2_block1_1_bn)
  conv2_block1_2_conv = denseNet121.get_layer('conv2_block1_2_conv')(conv2_block1_1_relu)
  conv2_block1_concat = denseNet121.get_layer('conv2_block1_concat')([pool1,conv2_block1_2_conv])

  conv2_block2_0_bn = denseNet121.get_layer('conv2_block2_0_bn')(conv2_block1_concat)
  conv2_block2_0_relu = denseNet121.get_layer('conv2_block2_0_relu')(conv2_block2_0_bn)
  conv2_block2_1_conv = denseNet121.get_layer('conv2_block2_1_conv')(conv2_block2_0_relu)
  conv2_block2_1_bn = denseNet121.get_layer('conv2_block2_1_bn')(conv2_block2_1_conv)
  conv2_block2_1_relu = denseNet121.get_layer('conv2_block2_1_relu')(conv2_block2_1_bn)
  conv2_block2_2_conv = denseNet121.get_layer('conv2_block2_2_conv')(conv2_block2_1_relu)
  conv2_block2_concat = denseNet121.get_layer('conv2_block2_concat')([conv2_block1_concat,conv2_block2_2_conv])

  conv2_block3_0_bn = denseNet121.get_layer('conv2_block3_0_bn')(conv2_block2_concat)
  conv2_block3_0_relu = denseNet121.get_layer('conv2_block3_0_relu')(conv2_block3_0_bn)
  conv2_block3_1_conv = denseNet121.get_layer('conv2_block3_1_conv')(conv2_block3_0_relu)
  conv2_block3_1_bn = denseNet121.get_layer('conv2_block3_1_bn')(conv2_block3_1_conv)
  conv2_block3_1_relu = denseNet121.get_layer('conv2_block3_1_relu')(conv2_block3_1_bn)
  conv2_block3_2_conv = denseNet121.get_layer('conv2_block3_2_conv')(conv2_block3_1_relu)
  conv2_block3_concat = denseNet121.get_layer('conv2_block3_concat')([conv2_block2_concat,conv2_block3_2_conv])

  conv2_block4_0_bn = denseNet121.get_layer('conv2_block4_0_bn')(conv2_block3_concat)
  conv2_block4_0_relu = denseNet121.get_layer('conv2_block4_0_relu')(conv2_block4_0_bn)
  conv2_block4_1_conv = denseNet121.get_layer('conv2_block4_1_conv')(conv2_block4_0_relu)
  conv2_block4_1_bn = denseNet121.get_layer('conv2_block4_1_bn')(conv2_block4_1_conv)
  conv2_block4_1_relu = denseNet121.get_layer('conv2_block4_1_relu')(conv2_block4_1_bn)
  conv2_block4_2_conv = denseNet121.get_layer('conv2_block4_2_conv')(conv2_block4_1_relu)
  conv2_block4_concat = denseNet121.get_layer('conv2_block4_concat')([conv2_block3_concat,conv2_block4_2_conv])

  conv2_block5_0_bn = denseNet121.get_layer('conv2_block5_0_bn')(conv2_block4_concat)
  conv2_block5_0_relu = denseNet121.get_layer('conv2_block5_0_relu')(conv2_block5_0_bn)
  conv2_block5_1_conv = denseNet121.get_layer('conv2_block5_1_conv')(conv2_block5_0_relu)
  conv2_block5_1_bn = denseNet121.get_layer('conv2_block5_1_bn')(conv2_block5_1_conv)
  conv2_block5_1_relu = denseNet121.get_layer('conv2_block5_1_relu')(conv2_block5_1_bn)
  conv2_block5_2_conv = denseNet121.get_layer('conv2_block5_2_conv')(conv2_block5_1_relu)
  conv2_block5_concat = denseNet121.get_layer('conv2_block5_concat')([conv2_block4_concat,conv2_block5_2_conv])

  conv2_block6_0_bn = denseNet121.get_layer('conv2_block6_0_bn')(conv2_block5_concat)
  conv2_block6_0_relu = denseNet121.get_layer('conv2_block6_0_relu')(conv2_block6_0_bn)
  conv2_block6_1_conv = denseNet121.get_layer('conv2_block6_1_conv')(conv2_block6_0_relu)
  conv2_block6_1_bn = denseNet121.get_layer('conv2_block6_1_bn')(conv2_block6_1_conv)
  conv2_block6_1_relu = denseNet121.get_layer('conv2_block6_1_relu')(conv2_block6_1_bn)
  conv2_block6_2_conv = denseNet121.get_layer('conv2_block6_2_conv')(conv2_block6_1_relu)
  conv2_block6_concat = denseNet121.get_layer('conv2_block6_concat')([conv2_block5_concat,conv2_block6_2_conv])

  pool2_bn = denseNet121.get_layer('pool2_bn')(conv2_block6_concat)
  pool2_relu = denseNet121.get_layer('pool2_relu')(pool2_bn)
  pool2_conv = denseNet121.get_layer('pool2_conv')(pool2_relu)
  pool2_pool = denseNet121.get_layer('pool2_pool')(pool2_conv)
  #-------------------------------------------------------------------------
  conv3_block1_0_bn = denseNet121.get_layer('conv3_block1_0_bn')(pool2_pool)#concate with 128
  conv3_block1_0_relu = denseNet121.get_layer('conv3_block1_0_relu')(conv3_block1_0_bn)
  conv3_block1_1_conv = denseNet121.get_layer('conv3_block1_1_conv')(conv3_block1_0_relu)
  conv3_block1_1_bn = denseNet121.get_layer('conv3_block1_1_bn')(conv3_block1_1_conv)
  conv3_block1_1_relu = denseNet121.get_layer('conv3_block1_1_relu')(conv3_block1_1_bn)
  conv3_block1_2_conv = denseNet121.get_layer('conv3_block1_2_conv')(conv3_block1_1_relu)
  conv3_block1_concat = denseNet121.get_layer('conv3_block1_concat')([pool2_pool,conv3_block1_2_conv])

  conv3_block2_0_bn = denseNet121.get_layer('conv3_block2_0_bn')(conv3_block1_concat)
  conv3_block2_0_relu = denseNet121.get_layer('conv3_block2_0_relu')(conv3_block2_0_bn)
  conv3_block2_1_conv = denseNet121.get_layer('conv3_block2_1_conv')(conv3_block2_0_relu)
  conv3_block2_1_bn = denseNet121.get_layer('conv3_block2_1_bn')(conv3_block2_1_conv)
  conv3_block2_1_relu = denseNet121.get_layer('conv3_block2_1_relu')(conv3_block2_1_bn)
  conv3_block2_2_conv = denseNet121.get_layer('conv3_block2_2_conv')(conv3_block2_1_relu)
  conv3_block2_concat = denseNet121.get_layer('conv3_block2_concat')([conv3_block1_concat,conv3_block2_2_conv])

  conv3_block3_0_bn = denseNet121.get_layer('conv3_block3_0_bn')(conv3_block2_concat)
  conv3_block3_0_relu = denseNet121.get_layer('conv3_block3_0_relu')(conv3_block3_0_bn)
  conv3_block3_1_conv = denseNet121.get_layer('conv3_block3_1_conv')(conv3_block3_0_relu)
  conv3_block3_1_bn = denseNet121.get_layer('conv3_block3_1_bn')(conv3_block3_1_conv)
  conv3_block3_1_relu = denseNet121.get_layer('conv3_block3_1_relu')(conv3_block3_1_bn)
  conv3_block3_2_conv = denseNet121.get_layer('conv3_block3_2_conv')(conv3_block3_1_relu)
  conv3_block3_concat = denseNet121.get_layer('conv3_block3_concat')([conv3_block2_concat,conv3_block3_2_conv])

  conv3_block4_0_bn = denseNet121.get_layer('conv3_block4_0_bn')(conv3_block3_concat)
  conv3_block4_0_relu = denseNet121.get_layer('conv3_block4_0_relu')(conv3_block4_0_bn)
  conv3_block4_1_conv = denseNet121.get_layer('conv3_block4_1_conv')(conv3_block4_0_relu)
  conv3_block4_1_bn = denseNet121.get_layer('conv3_block4_1_bn')(conv3_block4_1_conv)
  conv3_block4_1_relu = denseNet121.get_layer('conv3_block4_1_relu')(conv3_block4_1_bn)
  conv3_block4_2_conv = denseNet121.get_layer('conv3_block4_2_conv')(conv3_block4_1_relu)
  conv3_block4_concat = denseNet121.get_layer('conv3_block4_concat')([conv3_block3_concat,conv3_block4_2_conv])

  conv3_block5_0_bn = denseNet121.get_layer('conv3_block5_0_bn')(conv3_block4_concat)
  conv3_block5_0_relu = denseNet121.get_layer('conv3_block5_0_relu')(conv3_block5_0_bn)
  conv3_block5_1_conv = denseNet121.get_layer('conv3_block5_1_conv')(conv3_block5_0_relu)
  conv3_block5_1_bn = denseNet121.get_layer('conv3_block5_1_bn')(conv3_block5_1_conv)
  conv3_block5_1_relu = denseNet121.get_layer('conv3_block5_1_relu')(conv3_block5_1_bn)
  conv3_block5_2_conv = denseNet121.get_layer('conv3_block5_2_conv')(conv3_block5_1_relu)
  conv3_block5_concat = denseNet121.get_layer('conv3_block5_concat')([conv3_block4_concat,conv3_block5_2_conv])

  conv3_block6_0_bn = denseNet121.get_layer('conv3_block6_0_bn')(conv3_block5_concat)
  conv3_block6_0_relu = denseNet121.get_layer('conv3_block6_0_relu')(conv3_block6_0_bn)
  conv3_block6_1_conv = denseNet121.get_layer('conv3_block6_1_conv')(conv3_block6_0_relu)
  conv3_block6_1_bn = denseNet121.get_layer('conv3_block6_1_bn')(conv3_block6_1_conv)
  conv3_block6_1_relu = denseNet121.get_layer('conv3_block6_1_relu')(conv3_block6_1_bn)
  conv3_block6_2_conv = denseNet121.get_layer('conv3_block6_2_conv')(conv3_block6_1_relu)
  conv3_block6_concat = denseNet121.get_layer('conv3_block6_concat')([conv3_block5_concat,conv3_block6_2_conv])

  conv3_block7_0_bn = denseNet121.get_layer('conv3_block7_0_bn')(conv3_block6_concat)
  conv3_block7_0_relu = denseNet121.get_layer('conv3_block7_0_relu')(conv3_block7_0_bn)
  conv3_block7_1_conv = denseNet121.get_layer('conv3_block7_1_conv')(conv3_block7_0_relu)
  conv3_block7_1_bn = denseNet121.get_layer('conv3_block7_1_bn')(conv3_block7_1_conv)
  conv3_block7_1_relu = denseNet121.get_layer('conv3_block7_1_relu')(conv3_block7_1_bn)
  conv3_block7_2_conv = denseNet121.get_layer('conv3_block7_2_conv')(conv3_block7_1_relu)
  conv3_block7_concat = denseNet121.get_layer('conv3_block7_concat')([conv3_block6_concat,conv3_block7_2_conv])

  conv3_block8_0_bn = denseNet121.get_layer('conv3_block8_0_bn')(conv3_block7_concat)
  conv3_block8_0_relu = denseNet121.get_layer('conv3_block8_0_relu')(conv3_block8_0_bn)
  conv3_block8_1_conv = denseNet121.get_layer('conv3_block8_1_conv')(conv3_block8_0_relu)
  conv3_block8_1_bn = denseNet121.get_layer('conv3_block8_1_bn')(conv3_block8_1_conv)
  conv3_block8_1_relu = denseNet121.get_layer('conv3_block8_1_relu')(conv3_block8_1_bn)
  conv3_block8_2_conv = denseNet121.get_layer('conv3_block8_2_conv')(conv3_block8_1_relu)
  conv3_block8_concat = denseNet121.get_layer('conv3_block8_concat')([conv3_block7_concat,conv3_block8_2_conv])

  conv3_block9_0_bn = denseNet121.get_layer('conv3_block9_0_bn')(conv3_block8_concat)
  conv3_block9_0_relu = denseNet121.get_layer('conv3_block9_0_relu')(conv3_block9_0_bn)
  conv3_block9_1_conv = denseNet121.get_layer('conv3_block9_1_conv')(conv3_block9_0_relu)
  conv3_block9_1_bn = denseNet121.get_layer('conv3_block9_1_bn')(conv3_block9_1_conv)
  conv3_block9_1_relu = denseNet121.get_layer('conv3_block9_1_relu')(conv3_block9_1_bn)
  conv3_block9_2_conv = denseNet121.get_layer('conv3_block9_2_conv')(conv3_block9_1_relu)
  conv3_block9_concat = denseNet121.get_layer('conv3_block9_concat')([conv3_block8_concat,conv3_block9_2_conv])

  conv3_block10_0_bn = denseNet121.get_layer('conv3_block10_0_bn')(conv3_block9_concat)
  conv3_block10_0_relu = denseNet121.get_layer('conv3_block10_0_relu')(conv3_block10_0_bn)
  conv3_block10_1_conv = denseNet121.get_layer('conv3_block10_1_conv')(conv3_block10_0_relu)
  conv3_block10_1_bn = denseNet121.get_layer('conv3_block10_1_bn')(conv3_block10_1_conv)
  conv3_block10_1_relu = denseNet121.get_layer('conv3_block10_1_relu')(conv3_block10_1_bn)
  conv3_block10_2_conv = denseNet121.get_layer('conv3_block10_2_conv')(conv3_block10_1_relu)
  conv3_block10_concat = denseNet121.get_layer('conv3_block10_concat')([conv3_block9_concat,conv3_block10_2_conv])

  conv3_block11_0_bn = denseNet121.get_layer('conv3_block11_0_bn')(conv3_block10_concat)
  conv3_block11_0_relu = denseNet121.get_layer('conv3_block11_0_relu')(conv3_block11_0_bn)
  conv3_block11_1_conv = denseNet121.get_layer('conv3_block11_1_conv')(conv3_block11_0_relu)
  conv3_block11_1_bn = denseNet121.get_layer('conv3_block11_1_bn')(conv3_block11_1_conv)
  conv3_block11_1_relu = denseNet121.get_layer('conv3_block11_1_relu')(conv3_block11_1_bn)
  conv3_block11_2_conv = denseNet121.get_layer('conv3_block11_2_conv')(conv3_block11_1_relu)
  conv3_block11_concat = denseNet121.get_layer('conv3_block11_concat')([conv3_block10_concat,conv3_block11_2_conv])

  conv3_block12_0_bn = denseNet121.get_layer('conv3_block12_0_bn')(conv3_block11_concat)
  conv3_block12_0_relu = denseNet121.get_layer('conv3_block12_0_relu')(conv3_block12_0_bn)
  conv3_block12_1_conv = denseNet121.get_layer('conv3_block12_1_conv')(conv3_block12_0_relu)
  conv3_block12_1_bn = denseNet121.get_layer('conv3_block12_1_bn')(conv3_block12_1_conv)
  conv3_block12_1_relu = denseNet121.get_layer('conv3_block12_1_relu')(conv3_block12_1_bn)
  conv3_block12_2_conv = denseNet121.get_layer('conv3_block12_2_conv')(conv3_block12_1_relu)
  conv3_block12_concat = denseNet121.get_layer('conv3_block12_concat')([conv3_block11_concat,conv3_block12_2_conv])

  pool3_bn = denseNet121.get_layer('pool3_bn')(conv3_block12_concat)
  pool3_relu = denseNet121.get_layer('pool3_relu')(pool3_bn)
  pool3_conv = denseNet121.get_layer('pool3_conv')(pool3_relu)
  pool3_pool = denseNet121.get_layer('pool3_pool')(pool3_conv)
  #-------------------------------------------------------------------------

  conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3_pool)
  conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
  drop5 = Dropout(0.5)(conv5)

  up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
  merge6 = concatenate([pool3_conv,up6], axis = 3)
  conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
  conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

  up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
  merge7 = concatenate([pool2_conv,up7], axis = 3)
  conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
  conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

  up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
  merge8 = concatenate([conv1relu,up8], axis = 3)
  conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
  conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

  up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
  # merge9 = concatenate([conv1,up9], axis = 3)
  conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(up9)
  conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

  model = Model(inputs, conv10)
  return model

In [ ]:
# denseNet169 = DenseNet169(include_top=False,input_shape=(512,512,3),pooling=None)
# denseNet169.trainable = False
# denseNet169_pretrained =  Model(denseNet169.input,denseNet169.get_layer('block1_conv2').output)

51888128/51877672 [==============================] - 1s 0us/step


In [ ]:
# denseNet201 = DenseNet201(include_top=False,input_shape=(512,512,3),pooling=None)
# denseNet201.trainable = False
# denseNet201_pretrained =  Model(denseNet201.input,denseNet201.get_layer('zero_padding2d_4').output)

In [ ]:
# mobileNetV2 = MobileNetV2(include_top=False,input_shape=(224,224,3),pooling=None)
# mobileNetV2.trainable = False
# mobileNetV2_pretrained =  Model(mobileNetV2.input,mobileNetV2.get_layer('Conv1').output)

9420800/9406464 [==============================] - 0s 0us/step


In [ ]:
# efficientNetV2B0 = EfficientNetV2B0(include_top=False,input_shape=(512,512,3),pooling=None)
# efficientNetV2B0.trainable = False
# efficientNetV2B0_pretrained =  Model(efficientNetV2B0.input,efficientNetV2B0.get_layer('normalization_3').output)

24289280/24274472 [==============================] - 1s 0us/step


In [ ]:

def  UnetEfficientNetV2B0(inputs):
  efficientNetV2B0 = EfficientNetV2B0(include_top=False,input_shape=(512,512,3),pooling=None)
  efficientNetV2B0.trainable = False
  rescaling = efficientNetV2B0.get_layer('rescaling')(inputs)
  normalization = efficientNetV2B0.get_layer('normalization')(rescaling)
  stem_conv = efficientNetV2B0.get_layer('stem_conv')(normalization)
  stem_bn = efficientNetV2B0.get_layer('stem_bn')(stem_conv)
  stem_activation = efficientNetV2B0.get_layer('stem_activation')(stem_bn)
  block1a_project_conv = efficientNetV2B0.get_layer('block1a_project_conv')(stem_activation)
  block1a_project_bn = efficientNetV2B0.get_layer('block1a_project_bn')(block1a_project_conv)
  block1a_project_activation = efficientNetV2B0.get_layer('block1a_project_activation')(block1a_project_bn)
  #-------------------------------------------------------------------------
  block2a_expand_conv = efficientNetV2B0.get_layer('block2a_expand_conv')(block1a_project_activation)
  block2a_expand_bn = efficientNetV2B0.get_layer('block2a_expand_bn')(block2a_expand_conv)
  block2a_expand_activation = efficientNetV2B0.get_layer('block2a_expand_activation')(block2a_expand_bn)
  block2a_project_conv = efficientNetV2B0.get_layer('block2a_project_conv')(block2a_expand_activation)
  block2a_project_bn = efficientNetV2B0.get_layer('block2a_project_bn')(block2a_project_conv)
  block2b_expand_conv = efficientNetV2B0.get_layer('block2b_expand_conv')(block2a_project_bn)
  block2b_expand_bn = efficientNetV2B0.get_layer('block2b_expand_bn')(block2b_expand_conv)
  block2b_expand_activation = efficientNetV2B0.get_layer('block2b_expand_activation')(block2b_expand_bn)
  block2b_project_conv = efficientNetV2B0.get_layer('block2b_project_conv')(block2b_expand_activation)
  block2b_project_bn = efficientNetV2B0.get_layer('block2b_project_bn')(block2b_project_conv)
  block2b_add = efficientNetV2B0.get_layer('block2b_add')([block2b_project_bn,block2a_project_bn])
  #-------------------------------------------------------------------------
  block3a_expand_conv = efficientNetV2B0.get_layer('block3a_expand_conv')(block2b_add)
  block3a_expand_bn = efficientNetV2B0.get_layer('block3a_expand_bn')(block3a_expand_conv)
  block3a_expand_activation = efficientNetV2B0.get_layer('block3a_expand_activation')(block3a_expand_bn)
  block3a_project_conv = efficientNetV2B0.get_layer('block3a_project_conv')(block3a_expand_activation)
  block3a_project_bn = efficientNetV2B0.get_layer('block3a_project_bn')(block3a_project_conv)
  block3b_expand_conv = efficientNetV2B0.get_layer('block3b_expand_conv')(block3a_project_bn)
  block3b_expand_bn = efficientNetV2B0.get_layer('block3b_expand_bn')(block3b_expand_conv)
  block3b_expand_activation = efficientNetV2B0.get_layer('block3b_expand_activation')(block3b_expand_bn)
  block3b_project_conv = efficientNetV2B0.get_layer('block3b_project_conv')(block3b_expand_activation)
  block3b_project_bn = efficientNetV2B0.get_layer('block3b_project_bn')(block3b_project_conv)
  block3b_add = efficientNetV2B0.get_layer('block3b_add')([block3b_project_bn,block3a_project_bn])
  #-------------------------------------------------------------------------
  block4a_expand_conv = efficientNetV2B0.get_layer('block4a_expand_conv')(block3b_add)
  block4a_expand_bn = efficientNetV2B0.get_layer('block4a_expand_bn')(block4a_expand_conv)
  block4a_expand_activation = efficientNetV2B0.get_layer('block4a_expand_activation')(block4a_expand_bn)
  #-------------------------------------------------------------------------
  maxpool = MaxPool2D()(block4a_expand_activation)

  conv5 = Conv2D(576, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(maxpool)
  conv5 = Conv2D(576, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
  drop5 = Dropout(0.5)(conv5)

  up6 = Conv2D(384, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
  merge6 = concatenate([block4a_expand_activation,up6], axis = 3)
  conv6 = Conv2D(384, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
  conv6 = Conv2D(384, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

  up7 = Conv2D(192, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
  merge7 = concatenate([block2b_add,up7], axis = 3)
  conv7 = Conv2D(192, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
  conv7 = Conv2D(192, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

  up8 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
  merge8 = concatenate([block1a_project_activation,up8], axis = 3)
  conv8 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
  conv8 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

  up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
  merge9 = concatenate([normalization,up9], axis = 3)
  conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(up9)
  conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

  model = Model(inputs, conv10)
  return model